# SQL query from table names - Continued

In [21]:
!pip install dotenv

In [43]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = ''

## The old Prompt

In [36]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [37]:
context_user = [{
    'role': 'system',
    'content': """
-- Table: employees
CREATE TABLE employees (
  employee_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  department_id INTEGER,
  position VARCHAR(50)
);
-- Sample Rows:
-- 1, 'John Doe', 10, 'Manager'
-- 2, 'Jane Smith', 20, 'Analyst'
-- 3, 'Alice Johnson', 20, 'Engineer'

-- Table: departments
CREATE TABLE departments (
  department_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  location VARCHAR(50)
);
-- Sample Rows:
-- 10, 'Sales', 'New York'
-- 20, 'Engineering', 'San Francisco'
-- 30, 'HR', 'Chicago'

-- Table: salaries
CREATE TABLE salaries (
  salary_id INTEGER PRIMARY KEY,
  employee_id INTEGER,
  salary DECIMAL(10,2),
  effective_date DATE
);
-- Sample Rows:
-- 1, 1, 80000.00, '2022-01-01'
-- 2, 2, 65000.00, '2022-01-01'
-- 3, 3, 70000.00, '2022-01-01'

"""
}]



In [38]:
context.append({
    'role': 'system',
    'content': """
-- Example Query 1:
-- Q: Which employee has the highest salary?
-- SQL:
SELECT e.name, s.salary
FROM employees e
JOIN salaries s ON e.employee_id = s.employee_id
ORDER BY s.salary DESC
LIMIT 1;

-- Example Query 2:
-- Q: Which department has the most employees?
-- SQL:
SELECT d.name, COUNT(e.employee_id) AS employee_count
FROM departments d
JOIN employees e ON d.department_id = e.department_id
GROUP BY d.department_id
ORDER BY employee_count DESC
LIMIT 1;

-- Example Query 3:
-- Q: List all employees with their department names and current salary.
-- SQL:
SELECT e.name, d.name AS department, s.salary
FROM employees e
JOIN departments d ON e.department_id = d.department_id
JOIN salaries s ON e.employee_id = s.employee_id;
"""
})

In [39]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [40]:
#new
context_user = context.copy()
print(return_CCRMSQL("""YOUR QUERY HERE""", context_user))

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [28]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [29]:
#new
print(return_CCRMSQL("YOUR QUERY HERE", context_user))

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [49]:
# Example Query 1: What is the average salary of employees in the Engineering department?
query1 = "What is the average salary of employees in the Engineering department?"

# Example Query 2: List the names and salaries of employees earning more than $70,000.
query2 = "List the names and salaries of employees earning more than 70000."

# Example Query 3: What is the total payroll for each department?
query3 = "What is the total payroll for each department?"

In [44]:
print(return_CCRMSQL(query1, context_user))

To find the average salary of employees in the Engineering department, you can use the following SQL query:

```sql
SELECT AVG(s.salary) AS average_salary
FROM employees e
JOIN salaries s ON e.employee_id = s.employee_id
JOIN departments d ON e.department_id = d.department_id
WHERE d.name = 'Engineering';
```


In [46]:
print(return_CCRMSQL(query1, old_context))

Something nice and simple, please ask about SQL!


In [47]:
print(return_CCRMSQL(query2, context_user))

```sql
SELECT e.name, s.salary
FROM employees e
JOIN salaries s ON e.employee_id = s.employee_id
WHERE s.salary > 70000;
```


In [45]:
print(return_CCRMSQL(query2, old_context))

This is your SQL:

```sql
SELECT e.name, s.salary 
FROM employees e 
JOIN salary s ON e.ID_usr = s.ID_usr 
WHERE s.salary > 70000;
```

This query retrieves the names and salaries of employees whose salary exceeds 70,000 by joining the `employees` and `salary` tables.


In [50]:
print(return_CCRMSQL(query3, context_user))

To calculate the total payroll for each department, you can sum the salaries of employees grouped by their department. Here is the SQL query to achieve that:

```sql
SELECT d.name AS department, SUM(s.salary) AS total_payroll
FROM departments d
JOIN employees e ON d.department_id = e.department_id
JOIN salaries s ON e.employee_id = s.employee_id
GROUP BY d.department_id;
``` 

This query joins the `departments`, `employees`, and `salaries` tables, sums the salaries for each department, and groups the results by department.


In [51]:
print(return_CCRMSQL(query3, old_context))

Something nice and simple, please ask about SQL commands.
